### 1. Read the data set

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as py
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
churn=pd.read_csv("/content/drive/My Drive/AIML/Colab_Notebook/Residency6/Project/Intro_NN/bank.csv")

churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### 2. Drop the columns which are unique for all users like IDs

In [0]:
churn.shape

(10000, 14)

In [0]:
nunique = churn.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 10000].index

In [0]:
nunique

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [0]:
cols_to_drop

Index(['RowNumber', 'CustomerId'], dtype='object')

In [0]:
churn_df = churn.drop(cols_to_drop, axis=1)

In [0]:
churn_df.columns

Index(['Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

### 3. Distinguish the feature and target set

In [0]:
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(3)
memory usage: 937.6+ KB


In [0]:
#check missing values 

print(" The missing values in data is :", churn_df.isnull().sum().sum(),'\n')

 The missing values in data is : 0 



In [0]:
print("There are {} numeric and {} categorical columns in bank churn data".format(churn_df.select_dtypes(include=[np.number]).shape[1],churn_df.select_dtypes(exclude=[np.number]).shape[1]))

There are 9 numeric and 3 categorical columns in bank churn data


In [0]:
X_ = churn_df.drop(['Surname','Exited'], axis=1) 
y_ = churn_df["Exited"]
X =  X_.to_numpy()
y =  y_.to_numpy()
print(X.shape,y.shape)

(10000, 10) (10000,)


**We dropped the Surname and there now two categorical data (Geography, Gender). we need to Encoding categorical data
 before splitting**

In [0]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

In [0]:
#Geography
X[:,1]

array(['France', 'Spain', 'France', ..., 'France', 'Germany', 'France'],
      dtype=object)

In [0]:
#Gender
X[:,2]

array(['Female', 'Female', 'Female', ..., 'Female', 'Male', 'Female'],
      dtype=object)

In [0]:
label_encoder_x_1 = LabelEncoder()
X[: , 2] = label_encoder_x_1.fit_transform(X[:,2])
transformer = ColumnTransformer(
    transformers=[
        ("Geography",        # Just a name
         OneHotEncoder(), # The transformer class
         [1]              # The column(s) to be applied on.
         )
    ],
    remainder='passthrough' # donot apply anything to the remaining columns
)
X = transformer.fit_transform(X.tolist())
X = X.astype('float64')

In [0]:
#Geography
X[:, 1]

array([0., 0., 0., ..., 0., 1., 0.])

In [0]:
#Gender
X[:,2]

array([0., 1., 0., ..., 0., 0., 0.])

In [0]:
X

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

### 4. Divide the data set into training and test sets

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 7)

In [0]:
print(X.shape,y.shape)

(10000, 12) (10000,)


### 5. Normalize the train and test data

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
X_train

array([[ 0.996257  , -0.57581067, -0.57600312, ...,  0.64376017,
        -1.03667198, -1.73674714],
       [ 0.996257  , -0.57581067, -0.57600312, ...,  0.64376017,
         0.96462528, -0.90580392],
       [-1.00375706, -0.57581067,  1.73610172, ...,  0.64376017,
         0.96462528,  0.14574389],
       ...,
       [ 0.996257  , -0.57581067, -0.57600312, ...,  0.64376017,
         0.96462528, -0.22385005],
       [-1.00375706, -0.57581067,  1.73610172, ...,  0.64376017,
        -1.03667198,  0.47691986],
       [ 0.996257  , -0.57581067, -0.57600312, ..., -1.55337352,
        -1.03667198,  0.583741  ]])

In [0]:
X_test

array([[-1.00375706, -0.57581067,  1.73610172, ..., -1.55337352,
        -1.03667198,  1.29665366],
       [ 0.996257  , -0.57581067, -0.57600312, ...,  0.64376017,
         0.96462528,  0.92906648],
       [ 0.996257  , -0.57581067, -0.57600312, ..., -1.55337352,
        -1.03667198,  0.29297414],
       ...,
       [ 0.996257  , -0.57581067, -0.57600312, ...,  0.64376017,
         0.96462528,  1.15338919],
       [-1.00375706,  1.73668197, -0.57600312, ...,  0.64376017,
         0.96462528, -1.39537546],
       [-1.00375706,  1.73668197, -0.57600312, ...,  0.64376017,
         0.96462528,  1.64617434]])

### 6. Initialize & build the model

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [0]:
tf.random.set_seed(42)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [0]:
#Initialize Sequential model
model = Sequential()

In [0]:
#Add input layer
model.add(Dense(6, input_dim = 12, kernel_initializer = 'uniform', activation='relu', name='input_layer'))


In [0]:
#Add hidden layer
model.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu', name='hidden_layer_1'))

In [0]:
#Add Output layer
model.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid', name='output_layer'))

In [0]:
#Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
print('Neural Network Model Summary: ')
print(model.summary())

Neural Network Model Summary: 
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Dense)          (None, 6)                 78        
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 6)                 42        
_________________________________________________________________
output_layer (Dense)         (None, 1)                 7         
Total params: 127
Trainable params: 127
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# Fitting the ANN
model.fit(X_train,Y_train, epochs=100,validation_data=(X_test,Y_test))

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 3s 347us/sample - loss: 0.5638 - accuracy: 0.7958 - val_loss: 0.4412 - val_accuracy: 0.7945
Epoch 2/100
8000/8000 [==============================] - 1s 87us/sample - loss: 0.4377 - accuracy: 0.7968 - val_loss: 0.4324 - val_accuracy: 0.7945
Epoch 3/100
8000/8000 [==============================] - 1s 83us/sample - loss: 0.4303 - accuracy: 0.7968 - val_loss: 0.4290 - val_accuracy: 0.7945
Epoch 4/100
8000/8000 [==============================] - 1s 79us/sample - loss: 0.4271 - accuracy: 0.7968 - val_loss: 0.4265 - val_accuracy: 0.7945
Epoch 5/100
8000/8000 [==============================] - 1s 81us/sample - loss: 0.4241 - accuracy: 0.7968 - val_loss: 0.4246 - val_accuracy: 0.7945
Epoch 6/100
8000/8000 [==============================] - 1s 82us/sample - loss: 0.4216 - accuracy: 0.7968 - val_loss: 0.4222 - val_accuracy: 0.7945
Epoch 7/100
8000/8000 [==============================] - 1s 83u

### 7. Predict the results using 0.5 as a threshold

In [0]:
X_test

array([[-1.00375706, -0.57581067,  1.73610172, ..., -1.55337352,
        -1.03667198,  1.29665366],
       [ 0.996257  , -0.57581067, -0.57600312, ...,  0.64376017,
         0.96462528,  0.92906648],
       [ 0.996257  , -0.57581067, -0.57600312, ..., -1.55337352,
        -1.03667198,  0.29297414],
       ...,
       [ 0.996257  , -0.57581067, -0.57600312, ...,  0.64376017,
         0.96462528,  1.15338919],
       [-1.00375706,  1.73668197, -0.57600312, ...,  0.64376017,
         0.96462528, -1.39537546],
       [-1.00375706,  1.73668197, -0.57600312, ...,  0.64376017,
         0.96462528,  1.64617434]])

In [0]:
Y_pred = model.predict(X_test)

In [0]:
Y_pred

array([[0.10619459],
       [0.1514288 ],
       [0.06950706],
       ...,
       [0.14021425],
       [0.16459152],
       [0.23515685]], dtype=float32)

In [0]:
Y_pred = (Y_pred > 0.5).astype(np.int)
Y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

### 8. Print the Accuracy score and confusion matrix

In [0]:
from sklearn import metrics
print(metrics.classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.85      0.97      0.90      1589
           1       0.71      0.32      0.44       411

    accuracy                           0.83      2000
   macro avg       0.78      0.64      0.67      2000
weighted avg       0.82      0.83      0.81      2000



In [0]:
 print("accuracy", metrics.accuracy_score(Y_test, Y_pred))

accuracy 0.833


In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)


In [0]:

print("confusion matrix")
print(cm)

confusion matrix
[[1535   54]
 [ 280  131]]
